In [1]:
import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt

/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
%%time
click_data = pd.read_hdf("../data/click_data.hdf", "click_data")

CPU times: user 10.7 s, sys: 10.7 s, total: 21.4 s
Wall time: 21.3 s


In [3]:
click_data

,app,attributed_time,channel,click_id,click_time,device,ip,is_attributed,os
0,3,NaN,379,NaN,2017-11-06 14:32:21,1,83230,0.0,13
1,3,NaN,379,NaN,2017-11-06 14:33:34,1,17357,0.0,19
2,3,NaN,379,NaN,2017-11-06 14:34:12,1,35810,0.0,13
3,14,NaN,478,NaN,2017-11-06 14:34:52,1,45745,0.0,13
4,3,NaN,379,NaN,2017-11-06 14:35:08,1,161007,0.0,13
5,3,NaN,379,NaN,2017-11-06 14:36:26,1,18787,0.0,16
6,3,NaN,379,NaN,2017-11-06 14:37:44,1,103022,0.0,23
7,3,NaN,379,NaN,2017-11-06 14:37:59,1,114221,0.0,19
8,3,NaN,379,NaN,2017-11-06 14:38:10,1,165970,0.0,13
9,64,NaN,459,NaN,2017-11-06 14:38:23,1,74544,0.0,22


In [5]:
ids = click_data[["ip", "device", "os"]]

In [9]:
%%time
ids["ip"] = ids["ip"].astype("object")
ids["os"] = ids["os"].astype("object")
ids["device"] = ids["device"].astype("object")

/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


CPU times: user 13.1 s, sys: 3.35 s, total: 16.5 s
Wall time: 16.4 s


/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
%%time
print(ids["ip"].max())
print(ids["os"].max())
print(ids["device"].max())

364778
956
4227
CPU times: user 41.6 s, sys: 1.52 s, total: 43.1 s
Wall time: 42.9 s


In [14]:
%%time
ids["ip_os_device"] = ids["ip"] * 1000 * 10000 + ids["os"] * 10000 + ids["device"]

CPU times: user 34.2 s, sys: 19 s, total: 53.2 s
Wall time: 53.1 s


/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
ids = ids[["ip_os_device"]]

In [17]:
ids

,ip_os_device
0,832300130001
1,173570190001
2,358100130001
3,457450130001
4,1610070130001
5,187870160001
6,1030220230001
7,1142210190001
8,1659700130001
9,745440220001


In [18]:
%%time
ids.to_hdf("../data/ip_os_device", "ip_os_device")

/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->integer,key->block0_values] [items->['ip_os_device']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


CPU times: user 24.9 s, sys: 3.65 s, total: 28.6 s
Wall time: 28.4 s
